In [1]:
from DB import AIModelHandler
from DB import DataFrameHandler
import requests, datetime
import pandas as pd
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def getRealtimePrice(isin):
    url = f'https://polling.finance.naver.com/api/realtime?query=SERVICE_ITEM:{isin}'
    response = requests.get(url, verify=False)
    if response.status_code == 200:
        data = response.json()
        result = data['result']['areas'][0]['datas'][0]['nv']
        time = data['result']['time']
        
        return {'TIME' : datetime.datetime.fromtimestamp(time / 1000).strftime('%Y-%m-%d %H:%M:%S'),'ISIN' : isin, 'PRICE' : int(result)}

    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None
    
def getIndex(input):
    kosdaq_url = f'http://ecos.bok.or.kr/api/StatisticSearch/A5P1FVP5VX625OV2NVTA/json/kr/1/100000/802Y001/D/20130101/20240101/0089000'
    rate_url = f'http://ecos.bok.or.kr/api/StatisticSearch/A5P1FVP5VX625OV2NVTA/json/kr/1/100000/722Y001/D/20130101/20240101/0101000'
    kospi_url = f'http://ecos.bok.or.kr/api/StatisticSearch/A5P1FVP5VX625OV2NVTA/json/kr/1/100000/802Y001/D/20130101/20240101/0001000'
    url = {
        'kosdaq_url' : kosdaq_url,
        'rate_url' : rate_url,
        'kospi_url' : kospi_url
    }
    code = {
        'kospi_url' : 1,
        'kosdaq_url' : 2,
        'rate_url' : 3
    }
    response = requests.get(url[input], verify=False)
    result = []
    time = []
    if response.status_code == 200:
        data = response.json()
        IDX_CODE = [code[input]] * data['StatisticSearch']['list_total_count']
        for i in range(data['StatisticSearch']['list_total_count']):
            result.append(data['StatisticSearch']['row'][i]['DATA_VALUE'])
            time.append(data['StatisticSearch']['row'][i]['TIME'])
        
        result = pd.DataFrame({'WORK_DT' : time, 'IDX_CODE' : IDX_CODE, 'VALUE' : result})
        result['WORK_DT'] = result['WORK_DT'].str[:4] + '-' + result['WORK_DT'].str[4:6] + '-' + result['WORK_DT'].str[6:8]
        result['WORK_DT'] = pd.to_datetime(result['WORK_DT'], format='%Y-%m-%d')
        result = pd.merge(result, DataFrameHandler.findItems('WORK_DATE', orderBy='DATE_LABEL'), how='inner')
        result['WORK_DT'] = result['WORK_DT'].dt.strftime('%Y-%m-%d')
        result.drop(columns=['DATE_LABEL'], inplace = True)
        return result

    else:
        print(f"Failed to fetch data from {url}. Status code: {response.status_code}")
        return None

In [2]:
DataFrameHandler.insertItems('DAILY_INDEX', getIndex('rate_url'))
DataFrameHandler.insertItems('DAILY_INDEX', getIndex('kospi_url'))
DataFrameHandler.insertItems('DAILY_INDEX', getIndex('kosdaq_url'))

In [42]:
getRealtimePrice('005930')

{'TIME': '2023-10-01 15:07:28', 'ISIN': '005930', 'PRICE': 68400}

In [41]:
result = []
stockSet = AIModelHandler.selectAllStock()
for isin in stockSet:
    result.append(getRealtimePrice(isin))
pd.DataFrame(result)

NameError: name 'stockSet' is not defined